In [62]:
import pandas as pd
import numpy as np
import os

In [63]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("ulrikthygepedersen/speed-dating")
print("Path to dataset files:", path)

Path to dataset files: /home/user1/.cache/kagglehub/datasets/ulrikthygepedersen/speed-dating/versions/1


In [64]:
df = pd.read_csv(path+'/speeddating.csv')
df.loc[:, (df.dtypes == 'object').values] = df.loc[:, (df.dtypes == 'object').values].map(lambda x: x.replace("b'", '').replace("'", ""))

In [65]:
summary_df = pd.DataFrame({
    'dtype': df.dtypes.astype(str),
    'unique_count': df.nunique(),
    'missing_count': df.isnull().sum(),
    'mean': df.mean(numeric_only=True),
    'std': df.std(numeric_only=True),
    'min': df.min(numeric_only=True),
    'max': df.max(numeric_only=True),
    'mode': df.mode().iloc[0],  # 최빈값
    'zero_count': (df == 0).sum()  # 0값 개수
})

In [ ]:
pd.set_option('display.max_rows', 1000)   
pd.set_option('display.max_columns', 2000)
summary_df = summary_df.T
summary_df

,age,age_o,ambition,ambition_partner,ambitous_o,ambtition_important,art,attractive,attractive_important,attractive_o,attractive_partner,clubbing,concerts,d_age,d_ambition,d_ambition_partner,d_ambitous_o,d_ambtition_important,d_art,d_attractive,d_attractive_important,d_attractive_o,d_attractive_partner,d_clubbing,d_concerts,d_d_age,d_dining,d_exercise,d_expected_happy_with_sd_people,d_expected_num_interested_in_me,d_expected_num_matches,d_funny,d_funny_important,d_funny_o,d_funny_partner,d_gaming,d_guess_prob_liked,d_hiking,d_importance_same_race,d_importance_same_religion,d_intellicence_important,d_intelligence,d_intelligence_o,d_intelligence_partner,d_interests_correlate,d_like,d_movies,d_museums,d_music,d_pref_o_ambitious,d_pref_o_attractive,d_pref_o_funny,d_pref_o_intelligence,d_pref_o_shared_interests,d_pref_o_sincere,d_reading,d_shared_interests_important,d_shared_interests_o,d_shared_interests_partner,d_shopping,d_sincere,d_sincere_important,d_sincere_partner,d_sinsere_o,d_sports,d_theater,d_tv,d_tvsports,d_yoga,decision,decision_o,dining,exercise,expected_happy_with_sd_people,expected_num_interested_in_me,expected_num_matches,field,funny,funny_important,funny_o,funny_partner,gaming,gender,guess_prob_liked,has_null,hiking,importance_same_race,importance_same_religion,intellicence_important,intelligence,intelligence_o,intelligence_partner,interests_correlate,like,match,met,movies,museums,music,pref_o_ambitious,pref_o_attractive,pref_o_funny,pref_o_intelligence,pref_o_shared_interests,pref_o_sincere,race,race_o,reading,samerace,shared_interests_important,shared_interests_o,shared_interests_partner,shopping,sincere,sincere_important,sincere_partner,sinsere_o,sports,theater,tv,tvsports,wave,yoga
dtype,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,object,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,object,object,float64,object,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
unique_count,24,24,9,15,15,82,11,9,94,18,17,11,11,35,3,3,3,3,3,3,3,3,3,3,3,4,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,3,2,2,10,10,10,18,17,260,8,71,17,16,12,2,19,1,11,11,10,65,9,17,17,155,18,2,7,10,11,10,82,94,71,65,85,78,6,6,11,2,85,15,15,10,9,78,14,14,10,11,10,10,21,11
missing_count,95,104,105,712,722,99,79,105,79,212,202,79,79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,79,79,101,6578,1173,0,105,89,360,350,79,0,309,0,79,79,79,79,105,306,296,158,240,0,375,79,79,79,107,89,98,89,129,89,0,0,79,0,121,1076,1067,79,105,79,277,287,79,79,79,79,0,79
mean,26.358928,26.364999,7.578388,6.777524,6.778409,10.682539,6.714544,7.084733,22.514632,6.190411,6.189995,5.745993,6.825401,4.185605,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7.783829,6.245813,5.534131,5.570556,3.207814,NaN,8.403965,17.457043,6.400599,6.400598,3.881191,NaN,5.207523,NaN,5.737077,3.784793,3.651645,20.265613,7.70446,7.369301,7.368597,0.19601,6.134087,NaN,0.049856,7.919629,6.985781,7.851066,10.685375,22.495347,17.459714,20.270759,11.84593,17.396867,NaN,NaN,7.678515,NaN,11.84511

In [67]:
summary_df.loc[:, (summary_df.loc['unique_count'] > 10) & (summary_df.loc['dtype'] == 'object')]

,field
dtype,object
unique_count,260
missing_count,0
mean,NaN
std,NaN
min,NaN
max,NaN
mode,Business
zero_count,0


In [68]:
df_t = df['match']

In [69]:
df_i = df[[col for col in df.columns if col not in ['match','decision', "decision_o"]]].copy()
df_i.shape

(8378, 120)

In [70]:
df.shape

(8378, 123)

In [71]:
def categorize_field(field):
    field_lower = str(field).lower()
    law_keywords = ['law', 'legal']
    business_keywords = ['business', 'finance', 'mba', 'marketing', 'management', 'consulting', 'real estate', 'private equity']
    science_keywords = ['engineering', 'biology', 'chemistry', 'physics', 'math', 'medicine', 'biomedical', 'neuroscience', 'statistics', 'informatics', 'environmental', 'ecology', 'genetics', 'microbiology', 'biotechnology', 'industrial', 'operations research']
    social_keywords = ['education', 'social work', 'psychology', 'public administration', 'political science', 'international affairs', 'sociology', 'anthropology', 'public health', 'health policy', 'human rights', 'history', 'communication', 'curriculum', 'teaching', 'school', 'counseling', 'nutrition', 'speech', 'policy', 'development', 'urban planning']
    arts_keywords = ['art', 'music', 'film', 'theater', 'philosophy', 'literature', 'creative writing', 'language', 'nonfiction', 'poetry', 'acting', 'drama', 'classics']

    if any(k in field_lower for k in law_keywords):
        return 'Law'
    elif any(k in field_lower for k in business_keywords):
        return 'Business/Finance'
    elif any(k in field_lower for k in science_keywords):
        return 'Science/Engineering'
    elif any(k in field_lower for k in social_keywords):
        return 'Social Sciences/Education'
    elif any(k in field_lower for k in arts_keywords):
        return 'Arts/Humanities/Other'
    else:
        return 'Other'

df_i['category'] = df_i['field'].apply(categorize_field)

In [72]:
df_i.shape

(8378, 121)

In [73]:
df_i = df_i.drop('field', axis=1)
df_i.shape

(8378, 120)

In [74]:
df_i_e = pd.get_dummies(
    df_i,
    prefix=None,
    prefix_sep='_',
    dummy_na=True,
    columns=None,
    sparse=False,
    drop_first=True,
    dtype=None
)


In [75]:
df_i_e.shape

(8378, 248)

In [76]:
df_t.shape

(8378,)

In [ ]:
import re
df_i_e.columns = [re.sub(r'[\[\]<>]', '_', str(col)) for col in df_i_e.columns]

In [78]:
# 1. 데이터 분할
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    df_i_e, df_t, test_size=0.2, random_state=42, stratify=df_t
)

# 2. 앙상블 모델 생성 및 학습
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier
from sklearn.ensemble import VotingClassifier

xgb_model = xgb.XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
lgb_model = lgb.LGBMClassifier(random_state=42)
cat_model = CatBoostClassifier(verbose=0, random_state=42)

ensemble_model = VotingClassifier(
    estimators=[('xgb', xgb_model), ('lgb', lgb_model), ('cat', cat_model)],
    voting='soft'
)

ensemble_model.fit(X_train, y_train)

/home/user1/miniconda3/envs/pandas/lib/python3.12/site-packages/xgboost/training.py:183: UserWarning: [16:58:00] WARNING: /workspace/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 1104, number of negative: 5598
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001517 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1993
[LightGBM] [Info] Number of data points in the train set: 6702, number of used features: 187
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.164727 -> initscore=-1.623469
[LightGBM] [Info] Start training from score -1.623469


,estimators,"[('xgb', ...), ('lgb', ...), ...]"
,voting,'soft'
,weights,None
,n_jobs,None
,flatten_transform,True
,verbose,False
,objective,'binary:logistic'
,base_score,None
,booster,None
,callbacks,None
,colsample_bylevel,None


In [79]:
# 3. 테스트셋 예측 및 평가
from sklearn.metrics import accuracy_score, classification_report
y_pred = ensemble_model.predict(X_test)
print("정확도:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

정확도: 0.8663484486873508
              precision    recall  f1-score   support

           0       0.89      0.95      0.92      1400
           1       0.65      0.42      0.51       276

    accuracy                           0.87      1676
   macro avg       0.77      0.69      0.71      1676
weighted avg       0.85      0.87      0.85      1676

